In [ ]:
## Author: David Luna 
## Date: 12/11/2021
## SISTEMIC
import sys
import numpy as np
from functools import partial
from pathlib import Path
import pandas as pd
from pathlib import Path
import concurrent.futures
import time
import psutil
import warnings
from tqdm.notebook import tqdm
import maad
from scipy.signal import spectrogram
from scipy.signal import welch

sys.path.append("/home/david/Documents/Codes/postgresql/")
sys.path.append("../utils/")

warnings.filterwarnings("ignore")

#from functions.rawpsd import *
#from functions.rawindex import *
#from functions.procindex import *
from utils import *
#from functions.cpsd import *
import indices as indices
#from torchaudio.transforms import Spectrogram

## Funciones

In [ ]:
def get_indices(name, processed=False, fs2=None):
    try:
        s, fs = torchaudio.load(str(name))
        if s.size()[0] != 1:
            s = torch.unsqueeze(s[0,:], 0)
               
        indc = []
        tamano_ventana = 512
        nfft = 512
        tipo_ventana ="hann",
        sobreposicion = 0
        
        if processed == True:
            if fs != fs2:
                s = F.resample(s,fs,fs2,resampling_method="kaiser_window")
                fs = fs2      
            
            
            s = s/torch.sqrt(torch.mean(s**2))
            
            Sxx, tn, fn, ext = maad.sound.spectrogram(s[0,:], fs, mode='amplitude')
            Sxx_power,_,_,_ = maad.sound.spectrogram(s[0,:], fs)

            ACIft_ = indices.ACIft(Sxx)
            ADI  = maad.features.acoustic_diversity_index(Sxx,fn,fmax=int(fs/2))
            BETA = maad.features.bioacoustics_index(Sxx,fn,flim=(2000, 8000) , R_compatible=None)
            M = maad.features.temporal_median(s[0,:],mode='hilbert')
            
            Sxx_power2 = np.mean(Sxx_power, axis=1)
            Sxx_power2 = Sxx_power2[fn<=24000]
            fn2 = fn[fn<=24000]

            NP = maad.features.number_of_peaks(Sxx_power2, fn2, slopes=6, min_freq_dist=100, display=False)
            Hf, _ = maad.features.frequency_entropy(Sxx_power2) 
            Ht = maad.features.temporal_entropy(s.numpy()[0,:], mode='hilbert')
            H = Ht*Hf
            
            AEI = maad.features.acoustic_eveness_index(Sxx,fn, fmax=int(fs/2))
            NDSI, _, _, _  = maad.features.soundscape_index(Sxx_power,fn, flim_bioPh=(2000, 8000), flim_antroPh=(0, 2000), R_compatible=None)
            indc.extend([str(name).split('/')[-1],str(name).split('/')[-2],ACIft_,ADI,BETA,M,NP,H,AEI,NDSI])

            return indc

        else:
            Sxx, tn, fn, ext = maad.sound.spectrogram(s[0,:], fs, mode='amplitude')
            Sxx_power,_,_,_ = maad.sound.spectrogram(s[0,:], fs)


            ACIft_ = indices.ACIft(Sxx)
            ADI  = maad.features.acoustic_diversity_index(Sxx,fn,fmax=int(fs/2))
            BETA = maad.features.bioacoustics_index(Sxx,fn,flim=(2000, 8000) , R_compatible=None)
            M = maad.features.temporal_median(s[0,:],mode='hilbert')
            NP = maad.features.number_of_peaks(Sxx_power, fn, slopes=6, min_freq_dist=100, display=False)
            Hf, _ = maad.features.frequency_entropy(Sxx_power) 
            Ht = maad.features.temporal_entropy(s.numpy()[0,:], mode='hilbert')
            H = Ht*Hf
            AEI = maad.features.acoustic_eveness_index(Sxx,fn,fmax=int(fs/2))
            NDSI, _, _, _  = maad.features.soundscape_index(Sxx_power,fn, flim_bioPh=(2000, 8000), flim_antroPh=(0, 2000), R_compatible=None )

            indc.extend([str(name).split('/')[-1],str(name).split('/')[-2],ACIft_,ADI,BETA,M,NP,H,AEI,NDSI])

            return indc

    except:
        return np.nan



def get_psd(name, processed=False, fs2=None):
    try:
        s, fs = torchaudio.load(str(name))
        if s.size()[0] != 1:
            s = torch.unsqueeze(s[0,:], 0)
        if fs != fs2 and processed == True:
            s = F.resample(s,fs,fs2,resampling_method="kaiser_window")
            fs = fs2
        if fs == 48000:
            freq, Pxx = welch(s.numpy()[0], fs, nfft=512, noverlap=0)
        elif fs == 44100:
            freq, Pxx = welch(s, fs, nfft=512, noverlap=0)

        return [str(name).split('/')[-1], str(name).split('/')[-2], Pxx, freq]

    except:
        return welch(s.numpy()[0], fs, nfft=512, noverlap=0)


def get_spectrogram(name, processed=False, fs2=None):
    try:
        s, fs = torchaudio.load(str(name))
        if s.size()[0] != 1:
            s = torch.unsqueeze(s[0,:], 0)
        
        if fs != fs2 and processed == True:
            s = F.resample(s,fs,fs2,resampling_method="kaiser_window")
            fs = fs2
                

        f, t, Sxx = spectrogram(x=s, fs=fs,nfft=512, scaling='spectrum',mode ='magnitude', noverlap=0)
        return [str(name).split('/')[-1], str(name).split('/')[-2], Sxx, t, f]

    except:
        return np.nan

### Calcular indices sin procesar

In [ ]:
path = Path('/media/david/hdd/datasets/San Miguel')
files = list(path.rglob("*.[wW][aA][vV]"))
folder = str(path).split('/')[-1]
workers = psutil.cpu_count(logical=True) -2
fun = partial(get_indices,processed=False, fs2=None)
count = 0

print(f'Number files: {len(files)}')

list_ind = []
with concurrent.futures.ProcessPoolExecutor(max_workers=workers) as executor:
    start = time.time()
    for number, indics in zip(files, executor.map(fun, files)):
        list_ind.append(indics)
        count += 1
        
    stop = time.time()
    elapsed = stop - start
    print(f'Tiempo: {np.round(elapsed,2)} [Seg]')
    df = pd.DataFrame(columns=["name","folder","ACIft","ADI","BETA","M","NP","H","AEI","NDSI"], data=np.array(list_ind))
    df.to_excel(f'/home/david/Documentos/Codes/codes-indices-recorders/results/{folder}_indices.xlsx')

### Calcular indices procesando

In [ ]:
path = Path('/media/david/hdd/datasets/San Miguel')
files = list(path.rglob("*.[wW][aA][vV]"))
folder = str(path).split('/')[-1]
workers = psutil.cpu_count(logical=True)
fun = partial(get_indices,processed=True, fs2=48000)
count = 0

list_ind = []


print(f'Number files: {len(files)}')

start = time.time()
for f in tqdm(files):
    indics = fun(f)
    list_ind.append(indics)
    stop = time.time()
    elapsed = stop - start
print(f'Tiempo: {np.round(elapsed,2)} [Seg]')
    
df = pd.DataFrame(columns=["name","folder","ACIft","ADI","BETA","M","NP","H","AEI","NDSI"], data=np.array(list_ind))
df.to_excel(f'/home/david/Documentos/Codes/codes-indices-recorders/results/{folder}_indices_processed.xlsx')


# PSD raw

In [ ]:
path = Path('/media/david/hdd/datasets/San Miguel')
files = list(path.rglob("*.[wW][aA][vV]"))
folder = str(path).split('/')[-1]
workers = psutil.cpu_count(logical=True) -3
fun = partial(get_psd,processed=False, fs2=None)

print(len(files))
df = {'columns':["name","folder","PSD","freq"]}
temp = []

with concurrent.futures.ProcessPoolExecutor(max_workers=workers) as executor:
    start = time.time()
    count = 0
    for number, psds in tqdm(zip(files, executor.map(fun, files)),total=len(files)):
        temp.append(psds)
        count += 1
    stop = time.time()
    elapsed = stop - start
    print(f'Tiempo: {np.round(elapsed,2)} [Seg]')
    df['data'] = temp
    torch.save(df,f'{folder}_psd.pth')


# PSD Filtrada

In [ ]:
path = Path('/media/david/7316-1D94/datasets/camilo')
files = list(path.rglob("*.[wW][aA][vV]"))
folder = str(path).split('/')[-1]
workers = psutil.cpu_count(logical=True) -1
fun = partial(get_psd,processed=True, fs2=44100)


df = {'columns':["name","folder","PSD","freq"]}
temp = []

with concurrent.futures.ProcessPoolExecutor(max_workers=workers) as executor:
    start = time.time()
    count = 0
    for number, psds in tqdm(zip(files, executor.map(fun, files)),total=len(files)):
        temp.append(psds)
        count += 1
    stop = time.time()
    elapsed = stop - start
    print(f'Tiempo: {np.round(elapsed,2)} [Seg]')
    df['data'] = temp
    torch.save(df,f'{folder}_psd_processed.pth')

# Espectrograma promedio sin procesar

In [ ]:
path = Path('/media/david/7316-1D94/datasets/camilo')
files = list(path.rglob("*.[wW][aA][vV]"))
folder = str(path).split('/')[-1]
workers = psutil.cpu_count(logical=True) -3
fun = partial(get_spectrogram,processed=False, fs2=None)


df = {'columns':["name","folder","Sxx",'time','freq']}
temp = []

spec_prom = {
    'spec_G21A':0,
    'spec_G60A':0,
    'spec_S1A':0,
    'spec_S2A':0,
    'spec_G21S':0,
    'spec_G60S':0,
    'spec_S1S':0,
    'spec_S2S':0,
    'count_G21A':0,
    'count_G60A':0,
    'count_S1A':0,
    'count_S2A':0,
    'count_G21S':0,
    'count_G60S':0,
    'count_S1S':0,
    'count_S2S':0,
}

for x in tqdm(list(files)):
    data = get_spectrogram(str(x), processed=False, fs2=None)
    spec_prom[f'spec_{data[1]}'] += data[2]
    spec_prom[f'count_{data[1]}'] += 1
    spec_prom[f'time_{data[1]}'] = data[3]
    spec_prom[f'freq_{data[1]}'] = data[4]
    
   
torch.save(spec_prom,f'{folder}_spectrogram.pth')
